This is the most advanced notebook having models of our project. It encompasses the techniques of data augmentation, through chat GPT and also through, a list of synonyms that we created (with the help of chat gpt).

In the section of the Flaubert model, you can choose at the begining of the code what data you want to use as data_subset, so as training data.


*   If you use training_data, you are using the generic data given on Kaggle.
*   If you use output_data, you are using the generic and the synonym replacement done by chat gpt.
*   If you are using output_data_final you are adding the paraphrasing on top.
*   And finally, if you are using augmented_data then it is the Kaggle data and the manual replacement of synonyms with the list.
Always make sure you have computed the corresponding section before doing that.



The code is set up initally to make the same predictions as we did in our best submissions with this method, so the 63.5% of the name "DA_full_FlauBERT_large_vs0_2_bs16_decay0_05_rs42_dacs_1.csv"

# Initialisation


In [2]:
!pip install datasets transformers huggingface_hub sacremoses
!apt-get install git-lfs
!pip install transformers[torch]
!pip install accelerate -U
!pip install torch sklearn


# Import required packages
from transformers import pipeline, DataCollatorWithPadding
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from google.colab import files
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from google.colab import files
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import pandas as pd
import numpy as np
import torch
import xgboost as xgb
import torchgen
import string
import nltk
import torch
import xgboost as xgb


torch.cuda.is_available()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cu

False

In [3]:
# URL to the raw CSV file on GitHub
url = 'https://raw.githubusercontent.com/msperand/Machine_Learning_Project/main/Data/sample_submission.csv'

# Read the CSV file into a DataFrame
sample_submission = pd.read_csv(url)

# Display the first few rows of the DataFrame
print(sample_submission)


        id difficulty
0        0         A1
1        1         A1
2        2         A1
3        3         A1
4        4         A1
...    ...        ...
1195  1195         A1
1196  1196         A1
1197  1197         A1
1198  1198         A1
1199  1199         A1

[1200 rows x 2 columns]


In [4]:
# URL to the raw CSV file on GitHub
url2 = 'https://raw.github.com/msperand/Machine_Learning_Project/main/Data/training_data.csv'

# Read the CSV file into a DataFrame
training_data = pd.read_csv(url2)

# Display the first few rows of the DataFrame
print(training_data)

        id                                           sentence difficulty
0        0  Les coûts kilométriques réels peuvent diverger...         C1
1        1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2        2  Le test de niveau en français est sur le site ...         A1
3        3           Est-ce que ton mari est aussi de Boston?         A1
4        4  Dans les écoles de commerce, dans les couloirs...         B1
...    ...                                                ...        ...
4795  4795  C'est pourquoi, il décida de remplacer les hab...         B2
4796  4796  Il avait une de ces pâleurs splendides qui don...         C1
4797  4797  Et le premier samedi de chaque mois, venez ren...         A2
4798  4798  Les coûts liés à la journalisation n'étant pas...         C2
4799  4799  Sur le sable, la mer haletait de toute la resp...         C2

[4800 rows x 3 columns]


In [5]:
# URL to the raw CSV file on GitHub
url3 = 'https://raw.github.com/msperand/Machine_Learning_Project/main/Data/unlabelled_test_data.csv'

# Read the CSV file into a DataFrame
test_data = pd.read_csv(url3)

# Display the first few rows of the DataFrame
print(test_data)

        id                                           sentence
0        0  Nous dûmes nous excuser des propos que nous eû...
1        1  Vous ne pouvez pas savoir le plaisir que j'ai ...
2        2  Et, paradoxalement, boire froid n'est pas la b...
3        3  Ce n'est pas étonnant, car c'est une saison my...
4        4  Le corps de Golo lui-même, d'une essence aussi...
...    ...                                                ...
1195  1195  C'est un phénomène qui trouve une accélération...
1196  1196  Je vais parler au serveur et voir si on peut d...
1197  1197  Il n'était pas comme tant de gens qui par pare...
1198  1198      Ils deviennent dangereux pour notre économie.
1199  1199  Son succès a généré beaucoup de réactions néga...

[1200 rows x 2 columns]


# Data Augmentation

In this section, we upload and merge the datasets that we created using Chatgpt. One was done with synonym replacement , the other through paraphrasing. We end up with 14'400 sentences, many of which are duplicate, mostly because of teh low ability of chat gpt to find synonyms in French. However, this technique works very well, so we decided to keep it.

In [6]:
def merge_datasets(training_data, augmented_file, fraction):
    """
    Merges a fraction of the augmented dataset with the existing training dataset.

    Parameters:
        training_data (pd.DataFrame): DataFrame containing the training data.
        augmented_file (str): File path to the augmented dataset CSV.
        fraction (float): Fraction of the augmented dataset to merge (between 0 and 1).

    Returns:
        pd.DataFrame: A new DataFrame containing the training data and the selected fraction of the augmented data.
    """
    # Load the augmented dataset
    augmented_data = pd.read_csv(augmented_file)

    # Sample the fraction of the augmented data
    sample_size = int(len(augmented_data) * fraction)
    augmented_sample = augmented_data.sample(n=sample_size, random_state = 11)

    # Concatenate the training data with the sampled augmented data
    combined_data = pd.concat([training_data, augmented_sample], ignore_index=True)

    return combined_data


In [7]:

# Example usage assuming 'training_data' is already a DataFrame:
# training_data = pd.read_csv('training_data.csv')  # Assuming this step is done elsewhere in your code

augmented_file_path = 'https://raw.githubusercontent.com/msperand/Machine_Learning_Project/main/Data/augmented_training_data_chat_synonym.csv'  # Specify the path to your augmented dataset
output_data = merge_datasets(training_data, augmented_file_path, 1)
print(output_data.head())


   id                                           sentence difficulty
0   0  Les coûts kilométriques réels peuvent diverger...         C1
1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2   2  Le test de niveau en français est sur le site ...         A1
3   3           Est-ce que ton mari est aussi de Boston?         A1
4   4  Dans les écoles de commerce, dans les couloirs...         B1


In [8]:
augmented_file_path_2 = 'https://raw.githubusercontent.com/msperand/Machine_Learning_Project/main/Data/enhanced_paraphrased_data.csv'  # Specify the path to your augmented dataset
output_data_final = merge_datasets(output_data, augmented_file_path_2, 1)
print(output_data_final.head())

   id                                           sentence difficulty
0   0  Les coûts kilométriques réels peuvent diverger...         C1
1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2   2  Le test de niveau en français est sur le site ...         A1
3   3           Est-ce que ton mari est aussi de Boston?         A1
4   4  Dans les écoles de commerce, dans les couloirs...         B1


In [9]:
output_data_final.shape

(14400, 3)

# Data Augmentation with a list of synonym

In [ ]:
import random
from collections import defaultdict

# Extensive list of words with their difficulty levels
corpus = [
    ("chat", "A1"), ("chien", "A1"), ("maison", "A2"), ("voiture", "A2"), ("ordinateur", "B1"),
    ("programme", "B1"), ("difficulté", "C1"), ("intelligence", "C1"), ("livre", "A1"), ("école", "A1"),
    ("pomme", "A1"), ("banane", "A1"), ("avion", "A2"), ("bicyclette", "A2"), ("université", "B1"),
    ("recherche", "B1"), ("philosophie", "C1"), ("mathématique", "C1"), ("musique", "A1"), ("film", "A1"),
    ("étoile", "A1"), ("lune", "A1"), ("soleil", "A2"), ("planète", "A2"), ("galaxie", "B1"),
    ("univers", "B1"), ("théorie", "C1"), ("science", "C1"), ("mer", "A1"), ("océan", "A1"),
    ("rivière", "A2"), ("lac", "A2"), ("montagne", "B1"), ("vallée", "B1"), ("désert", "C1"),
    ("forêt", "C1"), ("arbre", "A1"), ("fleur", "A1"), ("herbe", "A1"), ("jardin", "A1"),
    ("oiseau", "A1"), ("chaton", "A1"), ("chiot", "A1"), ("poisson", "A1"), ("tigre", "B1"),
    ("lion", "B1"), ("serpent", "B1"), ("éléphant", "B1"), ("girafe", "B1"), ("zèbre", "B1"),
    ("rhinocéros", "C1"), ("hippopotame", "C1"), ("panthère", "C1"), ("léopard", "C1"),
    ("maître", "A1"), ("étudiant", "A1"), ("professeur", "A2"), ("docteur", "A2"), ("ingénieur", "B1"),
    ("scientifique", "B1"), ("astronaute", "C1"), ("pompier", "C1"), ("policier", "A1"),
    ("infirmier", "A1"), ("boulanger", "A1"), ("boucher", "A2"), ("cuisinier", "A2"),
    ("serveur", "A2"), ("écrivain", "B1"), ("poète", "B1"), ("artiste", "B1"), ("musicien", "B1"),
    ("acteur", "B1"), ("chanteur", "B1"), ("danseur", "B1"), ("peintre", "C1"), ("sculpteur", "C1"),
    ("architecte", "C1"), ("photographe", "C1"), ("cinéaste", "C1"), ("vérité", "A1"),
    ("amitié", "A1"), ("amour", "A1"), ("joie", "A1"), ("tristesse", "A2"), ("colère", "A2"),
    ("peur", "A2"), ("courage", "B1"), ("espoir", "B1"), ("désespoir", "B1"), ("liberté", "B1"),
    ("égalité", "C1"), ("fraternité", "C1"), ("justice", "C1"), ("paix", "C1"), ("bâtiment", "A1"),
    ("vêtements", "A1"), ("chaussures", "A1"), ("téléphone", "A2"), ("ordinateur", "A2"),
    ("table", "A1"), ("chaise", "A1"), ("lit", "A1"), ("fenêtre", "A2"), ("porte", "A2"),
    ("toit", "B1"), ("mur", "B1"), ("sol", "B1"), ("plafond", "B1"), ("cuisine", "B1"),
    ("salle", "B1"), ("salon", "B1"), ("chambre", "B1"), ("bureau", "B1"), ("garage", "B1"),
    ("jardin", "B1"), ("balcon", "C1"), ("terrasse", "C1"), ("cheminée", "C1"), ("toiture", "C1"),
    ("sous-sol", "C1"), ("grenier", "C1"), ("escalier", "C1"), ("ascenseur", "C1"), ("couloir", "C1"),
    ("télévision", "A1"), ("radio", "A1"), ("livre", "A1"), ("magazine", "A1"), ("journal", "A1"),
    ("ordinateur", "A1"), ("téléphone", "A1"), ("smartphone", "A2"), ("tablette", "A2"),
    ("écran", "A2"), ("clavier", "A2"), ("souris", "A2"), ("imprimante", "A2"), ("scanner", "B1"),
    ("copieur", "B1"), ("modem", "B1"), ("routeur", "B1"), ("wifi", "B1"), ("internet", "B1"),
    ("courriel", "B1"), ("email", "B1"), ("site", "B1"), ("page", "B1"), ("web", "B1"),
    ("blog", "B1"), ("forum", "B1"), ("chat", "B1"), ("réseau", "B1"), ("social", "B1"),
    ("média", "B1"), ("application", "B1"), ("logiciel", "B1"), ("programme", "B1"), ("système", "B1")
]

# Manually defined synonyms for the extensive list of words
synonyms = {
    "chat": ["félin", "minou"],
    "chien": ["canidé", "toutou"],
    "maison": ["demeure", "habitation"],
    "voiture": ["automobile", "véhicule"],
    "ordinateur": ["PC", "machine"],
    "programme": ["logiciel", "application"],
    "difficulté": ["complexité", "obstacle"],
    "intelligence": ["cognition", "sagacité"],
    "livre": ["ouvrage", "bouquin"],
    "école": ["établissement", "institution"],
    "pomme": ["fruit", "pommette"],
    "banane": ["fruit", "plantain"],
    "avion": ["aéronef", "appareil"],
    "bicyclette": ["vélo", "cycle"],
    "université": ["faculté", "institution"],
    "recherche": ["investigation", "exploration"],
    "philosophie": ["pensée", "sagesse"],
    "mathématique": ["calcul", "arithmétique"],
    "musique": ["mélodie", "harmonie"],
    "film": ["cinéma", "pellicule"],
    "étoile": ["astre", "lueur"],
    "lune": ["satellite", "astre"],
    "soleil": ["astre", "étoile"],
    "planète": ["astre", "globe"],
    "galaxie": ["constellation", "nébuleuse"],
    "univers": ["cosmos", "infini"],
    "théorie": ["hypothèse", "concept"],
    "science": ["savoir", "connaissance"],
    "mer": ["océan", "étendue"],
    "océan": ["mer", "étendue"],
    "rivière": ["fleuve", "cours d'eau"],
    "lac": ["étang", "plan d'eau"],
    "montagne": ["sommet", "pic"],
    "vallée": ["dépression", "creux"],
    "désert": ["étendue", "aridité"],
    "forêt": ["bois", "jungle"],
    "arbre": ["plante", "végétal"],
    "fleur": ["floral", "bloom"],
    "herbe": ["gazon", "pelouse"],
    "jardin": ["parc", "terrain"],
    "oiseau": ["volatile", "piaf"],
    "chaton": ["minou", "félin"],
    "chiot": ["toutou", "canidé"],
    "poisson": ["poiscaille", "poiscaille"],
    "tigre": ["félin", "panthère"],
    "lion": ["roi des animaux", "félin"],
    "serpent": ["ophidien", "reptile"],
    "éléphant": ["pachyderme", "géant"],
    "girafe": ["cervidé", "animal"],
    "zèbre": ["animal", "rayé"],
    "rhinocéros": ["pachyderme", "animal"],
    "hippopotame": ["pachyderme", "animal"],
    "panthère": ["félin", "tigre"],
    "léopard": ["félin", "panthère"],
    "maître": ["professeur", "enseignant"],
    "étudiant": ["élève", "apprenant"],
    "professeur": ["enseignant", "instructeur"],
    "docteur": ["médecin", "praticien"],
    "ingénieur": ["technicien", "spécialiste"],
    "scientifique": ["chercheur", "érudit"],
    "astronaute": ["cosmonaute", "explorateur"],
    "pompier": ["sapeur", "secouriste"],
    "policier": ["gendarme", "agent"],
    "infirmier": ["soignant", "assistant médical"],
    "boulanger": ["pâtissier", "artisan"],
    "boucher": ["charcutier", "viandard"],
    "cuisinier": ["chef", "cordon-bleu"],
    "serveur": ["garçon", "commis"],
    "écrivain": ["auteur", "littérateur"],
    "poète": ["versificateur", "rimeur"],
    "artiste": ["créateur", "peintre"],
    "musicien": ["instrumentiste", "compositeur"],
    "acteur": ["comédien", "artiste"],
    "chanteur": ["vocaliste", "interprète"],
    "danseur": ["balletomane", "chorégraphe"],
    "peintre": ["plasticien", "artiste"],
    "sculpteur": ["modeleur", "statuaire"],
    "architecte": ["concepteur", "designer"],
    "photographe": ["cameraman", "documentariste"],
    "cinéaste": ["réalisateur", "metteur en scène"],
    "vérité": ["réalité", "exactitude"],
    "amitié": ["camaraderie", "compagnonnage"],
    "amour": ["affection", "tendresse"],
    "joie": ["bonheur", "contentement"],
    "tristesse": ["chagrin", "désolation"],
    "colère": ["fureur", "irritation"],
    "peur": ["crainte", "terreur"],
    "courage": ["bravoure", "audace"],
    "espoir": ["espérance", "optimisme"],
    "désespoir": ["désolation", "découragement"],
    "liberté": ["indépendance", "autonomie"],
    "égalité": ["équité", "justice"],
    "fraternité": ["solidarité", "camaraderie"],
    "justice": ["équité", "impartialité"],
    "paix": ["tranquillité", "harmonie"],
    "bâtiment": ["édifice", "construction"],
    "vêtements": ["habits", "tenues"],
    "chaussures": ["souliers", "godasses"],
    "téléphone": ["portable", "mobile"],
    "table": ["meuble", "bureau"],
    "chaise": ["siège", "tabouret"],
    "lit": ["couche", "sommeil"],
    "fenêtre": ["ouverture", "baie"],
    "porte": ["entrée", "issue"],
    "toit": ["couvrement", "abri"],
    "mur": ["paroi", "cloison"],
    "sol": ["terre", "terrain"],
    "plafond": ["ciel", "voute"],
    "cuisine": ["gastronomie", "culinaire"],
    "salle": ["pièce", "chambre"],
    "salon": ["séjour", "living"],
    "bureau": ["travail", "étude"],
    "garage": ["stationnement", "atelier"],
    "balcon": ["terrasse", "loggia"],
    "terrasse": ["balcon", "patio"],
    "cheminée": ["foyer", "âtre"],
    "toiture": ["couvrement", "toit"],
    "sous-sol": ["cave", "entresol"],
    "grenier": ["combles", "mezzanine"],
    "escalier": ["marches", "degré"],
    "ascenseur": ["élévateur", "monte-charge"],
    "couloir": ["corridor", "passage"],
    "télévision": ["TV", "télé"],
    "radio": ["TSF", "émetteur"],
    "magazine": ["revue", "périodique"],
    "journal": ["quotidien", "gazette"],
    "smartphone": ["téléphone intelligent", "mobile"],
    "tablette": ["PC", "ardoise"],
    "écran": ["moniteur", "affichage"],
    "clavier": ["pianotage", "dactylo"],
    "souris": ["mulot", "pointeur"],
    "imprimante": ["copieur", "machine à imprimer"],
    "scanner": ["numériseur", "lecteur"],
    "copieur": ["duplicateur", "reprographie"],
    "modem": ["routeur", "connecteur"],
    "routeur": ["commutateur", "connecteur"],
    "wifi": ["réseau sans fil", "connexion"],
    "internet": ["cyberspace", "toile"],
    "courriel": ["email", "message électronique"],
    "email": ["courriel", "message électronique"],
    "site": ["web", "page"],
    "page": ["site", "feuille"],
    "web": ["internet", "site"],
    "blog": ["journal en ligne", "site personnel"],
    "forum": ["discussion", "groupe"],
    "chat": ["discussion en ligne", "conversation"],
    "réseau": ["network", "connexion"],
    "social": ["réseautage", "interaction"],
    "média": ["médium", "communication"],
    "application": ["logiciel", "programme"],
    "logiciel": ["programme", "application"],
    "programme": ["logiciel", "application"],
    "système": ["réseau", "infrastructure"]
}

# Create a dictionary for synonyms with the same difficulty
synonym_dict = defaultdict(list)
for word, difficulty in corpus:
    for synonym in synonyms.get(word, []):
        synonym_dict[word].append((synonym, difficulty))

# Function for synonym replacement
def synonym_replacement(sentence, synonym_dict, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word in synonym_dict]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = [syn for syn, _ in synonym_dict[random_word]]
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

# Function to augment an entire dataset
def augment_dataset(data, synonym_dict, augmentations_per_sentence=1):
    augmented_data = data.copy()
    for _ in range(augmentations_per_sentence):
        augmented_sentences = [
            synonym_replacement(sentence, synonym_dict, 1) for sentence in data['sentence']
        ]
        augmented_subset = data.copy()
        augmented_subset['sentence'] = augmented_sentences
        augmented_data = pd.concat([augmented_data, augmented_subset])
    augmented_data.reset_index(drop=True, inplace=True)
    return augmented_data

# Augment the dataset
augmented_data = augment_dataset(training_data, synonym_dict, augmentations_per_sentence=2)



In [ ]:
augmented_data


,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1
...,...,...,...
14395,4795,"C'est pourquoi, il décida de remplacer les hab...",B2
14396,4796,Il avait une de ces pâleurs splendides qui don...,C1
14397,4797,"Et le premier samedi de chaque mois, venez ren...",A2
14398,4798,Les coûts liés à la journalisation n'étant pas...,C2


# FlauBERT-large-cased

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, get_scheduler
from sklearn.model_selection import train_test_split
from transformers import FlaubertModel, FlaubertTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch
from transformers import FlaubertTokenizer, FlaubertForSequenceClassification


# Assuming the data is loaded into DataFrame 'training_data'
#data_subset = training_data
#data_subset = output_data
data_subset = output_data_final
#data_subset = augmented_data

# Split the subset into training and validation sets
train_data, val_data = train_test_split(data_subset, test_size=0.2, random_state=42)

# Set a random seed for PyTorch
torch.manual_seed(42)

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['difficulty'])
val_labels = label_encoder.transform(val_data['difficulty'])

# Initialize the tokenizer
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_large_cased')
model = FlaubertForSequenceClassification.from_pretrained('flaubert/flaubert_large_cased', num_labels=len(label_encoder.classes_))

# Function to tokenize the data
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Tokenize the data
train_encodings = tokenize_function(train_data['sentence'].tolist())
val_encodings = tokenize_function(val_data['sentence'].tolist())

# Prepare datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Switching from 'bert-base-multilingual-cased' to 'bert-large-multilingual-cased'
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_large_cased')
model = FlaubertForSequenceClassification.from_pretrained('flaubert/flaubert_large_cased', num_labels=len(label_encoder.classes_))

from transformers import TrainingArguments

# Training arguments: Increasing the number of training epochs
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # 5 in baseline
    per_device_train_batch_size=16, #8 in baseline
    per_device_eval_batch_size=16, #16 in Baseline
    warmup_steps=500, # 500 in baseline
    weight_decay=0.05, # 0.01 in baseline
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)


# Trainer setup with a custom optimizer and scheduler
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Define a function to compute metrics if needed
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/896k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

<ipython-input-9-f2083f2e9aaa>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.171000,1.034055,0.577431,0.617856,0.577431,0.576000
2,0.826300,0.608880,0.773264,0.789014,0.773264,0.774372
3,0.343900,0.293873,0.909722,0.910142,0.909722,0.909715
4,0.189400,0.227209,0.957292,0.957685,0.957292,0.957183
5,0.138800,0.198051,0.971875,0.972041,0.971875,0.971885


<ipython-input-9-f2083f2e9aaa>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-f2083f2e9aaa>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-f2083f2e9aaa>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-f2083f2e9aaa>:47: UserWarning: To copy construct from a 

TrainOutput(global_step=3600, training_loss=0.6788036198996835, metrics={'train_runtime': 1289.9946, 'train_samples_per_second': 44.651, 'train_steps_per_second': 2.791, 'total_flos': 1.33735620575232e+16, 'train_loss': 0.6788036198996835, 'epoch': 5.0})

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Assuming the model and tokenizer are already loaded
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = BertForSequenceClassification.from_pretrained('./model_directory', num_labels=len(label_encoder.classes_))
model.eval()  # Set the model to evaluation mode

# Load the test data (assuming it's loaded into a DataFrame named test_data)
# test_data = pd.read_csv('path/to/your/test_data.csv')  # Uncomment if you need to load from a file

# Tokenize the test data
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

test_encodings = tokenize_data(test_data['sentence'].tolist())


# Create a DataLoader for the test data
class TestData(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestData(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=32)

# Predicting with the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
predictions = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# Convert numerical predictions back to labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Create DataFrame with IDs and predicted difficulties
submission_df = pd.DataFrame({'id': test_data['id'], 'difficulty': predicted_labels})

# Save predictions to a CSV file
submission_file_path = 'DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1.csv' #dacs = data audmentation chat synonym, full = dacs and paraphrasing
submission_df.to_csv(submission_file_path, index=False)



<ipython-input-10-f920a3d36839>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the model
model.save_pretrained('./DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_model')

# Save the tokenizer
tokenizer.save_pretrained('./DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer')


('./DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer/tokenizer_config.json',
 './DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer/special_tokens_map.json',
 './DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer/vocab.json',
 './DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer/merges.txt',
 './DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer/added_tokens.json')

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define paths to save the model and tokenizer
drive_model_path = '/content/drive/My Drive/DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_model'
drive_tokenizer_path = '/content/drive/My Drive/DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer'

# Save the model to Google Drive
model.save_pretrained(drive_model_path)

# Save the tokenizer to Google Drive
tokenizer.save_pretrained(drive_tokenizer_path)

# Zip the model directory
shutil.make_archive(drive_model_path, 'zip', drive_model_path)

# Zip the tokenizer directory
shutil.make_archive(drive_tokenizer_path, 'zip', drive_tokenizer_path)

print(f"Model and tokenizer saved to Google Drive. You can download them from your Google Drive at:")
print(f"Model path: {drive_model_path}.zip")
print(f"Tokenizer path: {drive_tokenizer_path}.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model and tokenizer saved to Google Drive. You can download them from your Google Drive at:
Model path: /content/drive/My Drive/DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_model.zip
Tokenizer path: /content/drive/My Drive/DA_full_FlauBERT_large_vs0_2_decay0_05_rs42_dacs_1_tokenizer.zip


This part allows us to download the model and tokenizer on our google drive. It is the only way we manage to store it, as we did not manage to put it on GitHub.